## Challenge: Show how seeing the likes is not a enough to determine if a video is of quality or not.

#### Step 1: Download package and CSV File

In [1]:
#Install package dplyr
install.packages("dplyr")


The downloaded binary packages are in
	/var/folders/bd/q77b2thn6g5c73qx7bbtpx5w0000gn/T//Rtmp6fpV4M/downloaded_packages


In [2]:
#Download the CSV file that was saved on my computer from Kaggle.
setwd("~/Downloads")
yt_data <- read.csv("USvideos.csv", stringsAsFactors = FALSE)
head(yt_data)

,video_id,trending_date,title,channel_title,category_id,publish_time,tags,views,likes,dislikes,comment_count,thumbnail_link,comments_disabled,ratings_disabled,video_error_or_removed,description
,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>
1,2kyS6SvSYSE,17.14.11,WE WANT TO TALK ABOUT OUR MARRIAGE,CaseyNeistat,22,2017-11-13T17:13:01.000Z,SHANtell martin,748374,57527,2966,15954,https://i.ytimg.com/vi/2kyS6SvSYSE/default.jpg,False,False,False,SHANTELL'S CHANNEL - https://www.youtube.com/shantellmartin\nCANDICE - https://www.lovebilly.com\n\nfilmed this video in 4k on this -- http://amzn.to/2sTDnRZ\nwith this lens -- http://amzn.to/2rUJOmD\nbig drone - http://tinyurl.com/h4ft3oy\nOTHER GEAR --- http://amzn.to/2o3GLX5\nSony CAMERA http://amzn.to/2nOBmnv\nOLD CAMERA; http://amzn.to/2o2cQBT\nMAIN LENS; http://amzn.to/2od5gBJ\nBIG SONY CAMERA; http://amzn.to/2nrdJRO\nBIG Canon CAMERA; http://tinyurl.com/jn4q4vz\nBENDY TRIPOD THING; http://tinyurl.com/gw3ylz2\nYOU NEED THIS FOR THE BENDY TRIPOD; http://tinyurl.com/j8mzzua\nWIDE LENS; http://tinyurl.com/jkfcm8t\nMORE EXPENSIVE WIDE LENS; http://tinyurl.com/zrdgtou\nSMALL CAMERA; http://tinyurl.com/hrrzhor\nMICROPHONE; http://tinyurl.com/zefm4jy\nOTHER MICROPHONE; http://tinyurl.com/jxgpj86\nOLD DRONE (cheaper but still great);http://tinyurl.com/zcfmnmd\n\nfollow me; on http://instagram.com/caseyneistat\non https://www.facebook.com/cneistat\non https://twitter.com/CaseyNeistat\n\namazing intro song by https://soundcloud.com/discoteeth\n\nad disclosure. THIS IS NOT AN AD. not selling or promoting anything. but samsung did produce the Shantell Video as a 'GALAXY PROJECT' which is an initiative that enables creators like Shantell and me to make projects we might otherwise not have the opportunity to make. hope that's clear. if not ask in the comments and i'll answer any specifics.
2,1ZAPwfrtAFY,17.14.11,The Trump Presidency: Last Week Tonight with John Oliver (HBO),LastWeekTonight,24,2017-11-13T07:30:00.000Z,last week tonight trump presidency|last week tonight donald trump|john oliver trump|donald trump,2418783,97185,6146,12703,https://i.ytimg.com/vi/1ZAPwfrtAFY/default.jpg,False,False,False,"One year after the presidential election, John Oliver discusses what we've learned so far and enlists our catheter cowboy to teach Donald Trump what he hasn't.\n\nConnect with Last Week Tonight online...\n\nSubscribe to the Last Week Tonight YouTube channel for more almost news as it almost happens: www.youtube.com/user/LastWeekTonight\n\nFind Last Week Tonight on Facebook like your mom would: http://Facebook.com/LastWeekTonight\n\nFollow us on Twitter for news about jokes and jokes about news: http://Twitter.com/LastWeekTonight\n\nVisit our official site for all that other stuff at once: http://www.hbo.com/lastweektonight"
3,5qpjK5DgCt4,17.14.11,"Racist Superman | Rudy Mancuso, King Bach & Lele Pons",Rudy Mancuso,23,2017-11-12T19:05:24.000Z,racist superman|rudy|mancuso|king|bach|racist|superman|love|rudy mancuso poo bear black white official music video|iphone x by pineapple|lelepons|hannahstocking|rudymancuso|inanna|anwar|sarkis|shots|shotsstudios|alesso|anitta|brazil|Getting My Driver's License | Lele Pons,3191434,146033,5339,8181,https://i.ytimg.com/vi/5qpjK5DgCt4/default.jpg,False,False,False,WATCH MY PREVIOUS VIDEO ▶ \n\nSUBSCRIBE ► https://www.youtube.com/channel/UC5jkXpfnBhlDjqh0ir5FsIQ?sub_confirmation=1\n\nTHANKS FOR WATCHING! LIKE & SUBSCRIBE FOR MORE VIDEOS!\n-----------------------------------------------------------\nFIND ME ON: \nInstagram | http://instagram.com/rudymancuso\nTwitter | http://twitter.com/rudymancuso\nFacebook | http://facebook.com/rudymancuso\n\nCAST: \nRudy Mancuso | http://youtube.com/c/rudymancuso\nLele Pons | http://youtube.com/c/lelepons\nKing Bach | https://youtube.com/user/BachelorsPadTv\n\nVideo Effects: \nCaleb Natale | https://instagram.com/calebnatale\n\nPA:\nPaulina Gregory\n\n\nShots Studios Channels:\nAlesso

In [3]:
#Review column names
colnames(yt_data)

[1] "video_id"               "trending_date"          "title"                 
 [4] "channel_title"          "category_id"            "publish_time"          
 [7] "tags"                   "views"                  "likes"                 
[10] "dislikes"               "comment_count"          "thumbnail_link"        
[13] "comments_disabled"      "ratings_disabled"       "video_error_or_removed"
[16] "description"

#### Step 2: clean the data 
#### 2a) remove unnecessary columns as well as omiting data with NaN.

In [4]:
#removing unnecessary columns

yt_data$channel_id <- NULL
yt_data$thumbnail_link <- NULL
yt_data$video_id <- NULL
yt_data$description <- NULL
yt_data$tags <- NULL
yt_data$video_error_or_removed <- NULL
yt_data$trending_date <- NULL
yt_data$category_id <- NULL

#removes rows where NA values exist
yt_data <- na.omit(yt_data)

In [6]:
nrow(yt_data)

[1] 40949

##### Dataset has 40949 rows. 
##### 2b) Next part in the cleaning process is removing duplicate rows and rows where comments and likes are disabled. As well as adding my own columns in helpful for the analysis.

In [19]:

#remove data where the row is repeated
ct_data <- yt_data[!duplicated(yt_data[ ,c("title","publish_time", "channel_title")]), ]


#removes data that where like and dislike data is disabled.
yt_data1 <- ct_data$ratings_disabled == "False"
ct_data1 <- ct_data[yt_data1, ]

#removes data where comments are disabled.
ct_data2 <- ct_data1$comments_disabled == "False"
bt_data <- ct_data1[ct_data2, ]

#resets the row indices
rownames(bt_data) <- NULL

#remove columns
bt_data$comments_disabled <- NULL
bt_data$ratings_disabled <- NULL

#change dislikes from 0 to 1 so no NaN values
bt_data$dislikes[bt_data$dislikes == 0] <- 1

#change comment_count from 0 to 1 so no NaN values
bt_data$comment_count[bt_data$comment_count == 0] <- 1

#change likes from 0 to 1 so no NaN values
bt_data$likes[bt_data$likes == 0] <- 1

#adds column interaction to show views to likes 
bt_data$low_interaction<- bt_data$views / bt_data$likes

#adds column likes_ratio and shows the difference of likes to dislikes
bt_data$likes_diff <- bt_data$likes - bt_data$dislikes

#reordering data based on interaction
bt_data <- bt_data[order(-bt_data$low_interaction), ]
rownames(bt_data) <- NULL

### Completed: Cleaned data renamed bt_data where I can start to analyze it. 

In [20]:
nrow(bt_data)
head(bt_data)

[1] 6354

,title,channel_title,publish_time,views,likes,dislikes,comment_count,low_interaction,likes_diff
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,The Path Season 3 Trailer (Official) • The Path on Hulu,Hulu,2017-12-20T17:30:27.000Z,2159497,162,12,31,13330.228,150
2,THE SHAPE OF WATER | Human | FOX Searchlight,FoxSearchlight,2017-11-28T21:16:52.000Z,349383,38,2,4,9194.289,36
3,LuLaRoe Consultant Mocks Those With Special Needs,J S,2018-01-25T02:35:03.000Z,212858,27,591,209,7883.630,-564
4,Did Alexa Lose Her Voice? - Teaser - Amazon Super Bowl Commercial LII,amazon,2018-01-26T21:42:18.000Z,4683106,763,60,178,6137.754,703
5,"Michael Wolff, “Fire and Fury | theSkimm Sip 'n Skimm",theSkimm,2018-01-31T17:32:26.000Z,11216,2,14,13,5608.000,-12
6,Ref to the Rescue | 2018 Kona | Hyundai NFL Super Bowl LII,HyundaiUSA,2018-01-30T15:48:51.000Z,523481,100,13,9,5234.810,87


After the data is cleaned the total number of rows is 6354. Quite a lot of duplicates! Most videos had 5-8 duplicates when I was checking.
As seen above, the list of videos are ordered by descending low_interaction column (highly viewed videos but low-like-rating) to highest interaction. Does that mean these top videos are extremely disliked? Not necessarily, these videos display the dislikes as well and it can be seen that 4 out of the 6 top interaction videos have more likes to dislikes (view likes_diff). Meaning that just a low number of likes does not mean the video disliked. 

In [179]:
the_worst <- min(bt_data$likes_diff)
bt_data[which(bt_data$likes_diff == the_worst), ]

,title,channel_title,publish_time,views,likes,dislikes,comment_count,low_interaction,likes_diff
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
786,PSA from Chairman of the FCC Ajit Pai,Daily Caller,2017-12-13T22:52:57.000Z,1142585,9100,218841,28013,125.5588,-209741


   This is the most dislikes to likes video from the dataset.https://www.youtube.com/watch?v=LFhT6H6pRWg "PSA from Chairman of the FCC Ajit Pai" now as someone who had been on youtube for awhile I vaguely remember people hating on this person but did not remember who he was and watching this video without thinking about the dislikes, I thought he was just a meme guy. But looking at the number of dislikes, this person is more than just a goofball. He has been hated by lots of people https://www.wired.com/story/ajit-pai-man-who-killed-net-neutrality/and it can easily be seen in the like rating. This is critical information because how would I know if I stumble across other videos where hate is just as much. As a modern viewer 26 years of age, removing the ability to view the number of dislikes is poor behavior. 
     This is similar to voting most people can vote and not have to make a written opinion on it. Youtube could have dislikes still instilled. Who cares if small creators are recieving hate. According to this video, https://www.youtube.com/watch?v=TbI0xDKkNCYI others feel the same way. Removing dislikes disengages the viewer, leading them to misbelief and making them more oblivious to what is out there. I have had my share of following a how-to video on youtube without knowing if this person is actually knowledgable or creating dangers to the viewers who listen to him. Compare Youtube to Yelp. Yelp allows people to give a number of stars like 1 when the rating is bad. People can view the bad opinions and small businesses are not hurt because of it. These small business/creators know what they are getting themselves into when they publicize their creation. Youtube should be a free market. One where reviews can be shown of all types and ones where is suppose to stand on do not have expertise in who Ajit Pai is and according to Kaggle when this dataset was introduced dislikes were still visible. If this video was stumbled upon in August 2025, when I did this report it would show 12k likes and 1.7 million views this could just be a video with high views + 1 million and low interactions of likes and dislikes. 

In [185]:
row_dissed <- nrow(bt_data[bt_data$likes_diff < 0, ])
row_total <-nrow(bt_data)

In [187]:
print(paste0(round((row_dissed/row_total)*100, 2), "%"))

[1] "1.51%"


So total percent of videos where dislikes outweigh the likes is 1.51% now that means that there are about 1.5 videos for every 100 videos that are watched that give a negative taste to the majority. Now depending on what the average amount of videos watched a day is 1.5% could be a lot. A good analogy is like eating a worm out of an apple. https://www.oberlo.com/statistics/average-time-spent-on-youtube
https://www.mixbloom.com/resources/average-time-spent-on-social-media-2022 These data sources say people spend on average 4.6 videos a day thats every 4 days theres a rotten taste in their mouth.

# Next I would like to see the mean low_interaction and seperate the interactions based on the 25th percent quartile, 50th percent quartile and the 75th percent quatile to see how the like_diff is for these components.

In [23]:
#Standard deviation and coefficient of the variation
avg_int <- mean(bt_data$low_interaction)
sd_int <- sd(bt_data$low_interaction)
cof_var <- sd_int/avg_int
cof_var

[1] 3.986687

In [29]:
6354*.75
#nrow(bt_data[0:3177, ])

[1] 4765.5

In [109]:
#percent tile variables for the interaction group 
quarter1 <- bt_data[0:1588, ]
quarter2 <- bt_data[1558:3177, ]
quarter3 <- bt_data[3177:4765, ]
quarter4 <- bt_data[4765:6354, ]

In [111]:
#SD and CV of quarter one
avg1 <- mean(quarter1$low_interaction)
sd1 <- sd(quarter1$low_interaction)
cv1 <- sd1/avg1
avg_view1 <-mean(quarter1$views)
min_v1 <-min(quarter1$views)
max_v1 <-max(quarter1$views)

In [112]:
#SD and CV of quarter two
avg2 <- mean(quarter2$low_interaction)
sd2 <- sd(quarter2$low_interaction)
cv2 <- sd2/avg2
avg_view2 <-mean(quarter2$views)
min_v2 <- min(quarter2$views)
max_v2 <-max(quarter2$views)

In [113]:
#SD and CV of quarter three
avg3 <- mean(quarter3$low_interaction)
sd3 <- sd(quarter3$low_interaction)
cv3 <- sd3/avg3
avg_view3 <- mean(quarter3$views)
min_v3 <-min(quarter3$views)
max_v3 <-max(quarter3$views)

In [114]:
#SD and CV of quarter four
avg4 <- mean(quarter4$low_interaction)
sd4 <- sd(quarter4$low_interaction)
cv4 <- sd4/avg4
avg_view4 <- mean(quarter4$views)
min_v4 <-min(quarter4$views)
max_v4 <-max(quarter4$views)

In [122]:
df_int <- data.frame(
Min_view = c(min_v1, min_v2, min_v3, min_v4),
Max_view = c(max_v1, max_v2, max_v3, max_v4),
Mean_int = c(avg1, avg2, avg3, avg4),
SD_int = c(sd1, sd2, sd3, sd4), 
CV_int = c(cv1, cv2, cv3, cv4)
)
rownames(df_int) <- c("Quarter 1", "Quarter 2", "Quarter 3", "Quarter 4")
df_int$CV_int <- df_int$CV_int * 100
df_int$CV_int <- paste0(round(df_int$CV_int, 1), "%")
df_int$SD_int <- round(df_int$SD_int, 2)
df_int$Mean_int <- round(df_int$Mean_int, 2)
df_int$Max_view <- format(df_int$Max_view, big.mark = ",", scientific = FALSE)
df_int$Min_view <- format(df_int$Min_view, big.mark = ",", scientific = FALSE)
df_int

,Min_view,Max_view,Mean_int,SD_int,CV_int
,<chr>,<chr>,<dbl>,<dbl>,<chr>
Quarter 1,658,"48,431,654",265.58,661.03,248.9%
Quarter 2,549,"38,873,543",46.00,10.56,22.9%
Quarter 3,"1,087","37,736,281",24.01,3.91,16.3%
Quarter 4,798,"39,349,927",12.44,3.42,27.4%


So in this case I had seperated the data into quarter tiles and in each the mean, SD and CV is seen. Quarter 1 has high variance the interaction has data of all over. 

running the code below
quarter1[quarter1$likes_diff == 0, ] has equal likes to dislikes there isnt really anything that stands out from these videos. These videos do not share hate but have low interaction in them.

In [156]:
bt_data[bt_data$likes_diff < 0, ]

,title,channel_title,publish_time,views,likes,dislikes,comment_count,low_interaction,likes_diff
,<chr>,<chr>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
3,LuLaRoe Consultant Mocks Those With Special Needs,J S,2018-01-25T02:35:03.000Z,212858,27,591,209,7883.6296,-564
5,"Michael Wolff, “Fire and Fury | theSkimm Sip 'n Skimm",theSkimm,2018-01-31T17:32:26.000Z,11216,2,14,13,5608.0000,-12
8,Hope Detector | Hyundai NFL Super Bowl LII,HyundaiUSA,2018-02-04T23:31:40.000Z,1217201,243,600,105,5009.0576,-357
12,WEEI: Alex Reimer calls Tom Brady’s daughter an “Annoying Little Pissant” 1/25/18,Nick Ramos,2018-01-27T04:20:20.000Z,994747,258,398,629,3855.6085,-140
19,World's first self-driving shuttle crashes on first day of use,KSNV News 3 Las Vegas,2017-11-08T20:16:47.000Z,169511,57,59,79,2973.8772,-2
20,IoT kit robot #3,eba era,2014-03-07T22:39:36.000Z,2910,1,3,1,2910.0000,-2
26,MTB Skills - Technical Climb,Geoff Mallinson,2015-04-02T12:41:47.000Z,4681,2,3,1,2340.5000,-1
29,Kelly Oubre Punches John Wall in the Lead during warriors wizards scuffle,Rob Andretti,2017-10-28T11:15:14.000Z,2197,1,2,1,2197.0000,-1
34,Devin Booker has words with Lakers assistant Jesse Mermuys,NBA Highlights · YouTube,2017-11-15T17:29:26.000Z,10838,7,42,10,1548.2857,-35




## Challenge 2 Find videos with low like ratios under 1 and see the number of columns

In [ ]:
sum(yt_data$likes_ratio >= 100, na.rm = TRUE)

In [ ]:
#clipped out a list of how the data is for a video with no likes, comments, or dislikes
c <- is.nan(yt_data$likes_ratio)

Like this video

In [ ]:
yt_views <- yt_data[order(yt_data$interaction), ]
head(yt_views)

In [ ]:
yt_data[1071, ]

The dislikes have a total of 218,841 and a likes were at 9,100. By watching the video with no dislikes, I could not find this video to be informative. I figured this was another meme video and not a video where people felt oppressed or opinionated. Openly sharing the dislikes gives a better image of what others feel when watching a video.  

With my reign of searching I was able to find this as an outlet for viewing the dislike button. I would like to share this video for those that want to see the number of dislikes a video has again. https://www.youtube.com/watch?v=cUVGjT8szhQ&t=16s

In [ ]:
yt_data <- yt_data[rev(1:nrow(yt_data)), ]

In [ ]:
yt_data[(1:29), ]

In [ ]:
yt_time[which.max(yt_time$dislikes), ]